In [2]:
# inmpoerting the librery...............


from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
discription=[]
drug_class=[]
interaction=[]
side_effect=[]
how_use=[]
drugs=[]

In [3]:
# geting url of the website..........

url='https://www.drugs.com/drug_information.html'
response=get(url)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
#store all the alphabates so scrap all the drugs in alphabaticaly order
alpha=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [6]:
a_z_links=[]
drugs_name=[]

In [7]:
# geting all the drugs and link in alphabetically order
link='https://www.drugs.com'
for alpa in alpha:
    alpha_url='https://www.drugs.com/alpha/'+alpa+'.html'
    alpha_response=get(alpha_url)
    alpha_soup= BeautifulSoup(alpha_response.text, 'html.parser')
    for el in alpha_soup.find_all('ul' ,{'class': 'ddc-list-column-2'}):
        for i in el.find_all("li"):
            links = i.findAll('a')
            for a in links:
                a_z_links.append(link+a['href'])
                drugs_name.append(a.text)


In [8]:
#conver into dataframe 
d = {'Drugs_name':drugs_name,'Link':a_z_links}
df = pd.DataFrame(d)
df.head()

,Drugs_name,Link
0,Abilify,https://www.drugs.com/abilify.html
1,Acetaminophen,https://www.drugs.com/acetaminophen.html
2,Aciphex,https://www.drugs.com/aciphex.html
3,Actos,https://www.drugs.com/actos.html
4,Acyclovir,https://www.drugs.com/acyclovir.html


In [9]:
i=0
for drug_url1 in a_z_links:
    if(i==528):
        i+=1
        continue
    print(drug_url1)
    drug_url=drug_url1
    # retriving all the link ..............
    drug_response=get(drug_url)
    drug_soup= BeautifulSoup(drug_response.text, 'html.parser')
    #some link redirect at some other section(Professional) so from there we have to redirect at the Overview section
    #for example: https://www.drugs.com/pro/basaglar.html   to...     https://www.drugs.com/basaglar.html
    if(drug_soup.find('ul',{'class':"nav-tabs nav-tabs-collapse"})):
        test = drug_soup.find('ul',{'class':"nav-tabs nav-tabs-collapse"})
        if(test.find('li')):
            aa=test.find('li')
            if(aa.find('a')):
                #from Overview section(https://www.drugs.com/basaglar.html)
                print("FROM Professional section to Overview section ")
                link='https://www.drugs.com'+aa.find('a')['href']
                drug_url=link
                drug_response=get(drug_url)
                drug_soup= BeautifulSoup(drug_response.text, 'html.parser')
                print(link)
                
     # ..........Retriving all the header................
    header=[]
    header3=[]
    for headings in drug_soup.find_all('h2' ):
        header.append(headings.text)
    for headings in drug_soup.find_all('h3' ):
        header3.append(headings.text)
    print(i)
    i+=1
    print(len(header))
    print(len(header3))
    if(len(header)==12 or len(header)==13):
    # ................drugs name from drugs_name List.......................
    
    
        drugs.append(drugs_name[i-1])
        
        #..........# Retriving the Drigs Class.............
        
        if(drug_soup.find('div',{'class':"sideBox sideBoxDrugClass sideBoxFloatLeft"})):
            title = drug_soup.find('div',{'class':"sideBox sideBoxDrugClass sideBoxFloatLeft"})
            if(title.find('div', {'class':"sideBoxContent"})):
                name =title.find('div', {'class':"sideBoxContent"})
                drug_class.append(name.find('a').text)
                print(name.find('a').text)
            #........ #in some drugs link not aviliable the drugs class section : ................
            
            else:
                print(drugs_name[i-1])
                drug_class.append(drugs_name[i-1])
        else:
            print(drugs_name[i-1])
            drug_class.append(drugs_name[i-1])

            #......# Retriving the drugs discription (What is.........section).............
            
        final=''
        final_text=[]
        start = drug_soup.find('h2', text=header[0])
        end = drug_soup.find('h2', text=header[1])
        content = ''
        item = start.nextSibling

        while item != end:
            content += str(item)
            item = item.nextSibling
            #...#cleaning the text and removing the extra spaces
            content=re.sub('start 300 ad', '', content)
            content=re.sub('end 300 ad', '', content)
            content=re.sub('\n', '', content)
            content=content.strip()
            content=re.sub('<[^<]+?>', '', content)
            final=content
        final_text.append(final)
        discription.append(final_text)

    # retriving side effet of drugs section...............
        final=''
        final_text=[]
        start = drug_soup.find('h2', text=header[7])
        end = drug_soup.find('h2', text=header[8])
        content = ''
        item = start.nextSibling

        while item != end:
            content += str(item)
            item = item.nextSibling
            content=re.sub('start 300 ad', '', content)
            content=re.sub('end 300 ad', '', content)
            content=re.sub('\n', '', content)
            content=content.strip()
            content=re.sub('<[^<]+?>', ' ', content)
            final=content
        final_text.append(final)
        side_effect.append(final_text)

        #retriving the intrecation of dugs section
        
        final=''
        final_text=[]
        start = drug_soup.find('h2', text=header[8])
        end = drug_soup.find('h2', text=header[9])
        content = ''
        item = start.nextSibling

        while item != end:
            content += str(item)
            item = item.nextSibling
            content=re.sub('start 300 ad', '', content)
            content=re.sub('end 300 ad', '', content)
            content=re.sub('\n', '', content)
            content=content.strip()
            content=re.sub('<[^<]+?>', '', content)
            final=content
        final_text.append(final)
        interaction.append(final_text)


#retrivng How to Use drugs section.....
        final=''
        final_text=[]
        start = drug_soup.find('h2', text=header[3])
        end = drug_soup.find('h2', text=header[4])
        content = ''
        item = start.nextSibling

        while item != end:
            content += str(item)
            item = item.nextSibling
            content=re.sub('start 300 ad', '', content)
            content=re.sub('end 300 ad', '', content)
            content=re.sub('\n', '', content)
            content=content.strip()
            content=re.sub('<[^<]+?>', '', content)
            final=content
        final_text.append(final)
        how_use.append(final_text)    
    else:
        discription.append("null")
        drug_class.append("null")
        interaction.append("null")
        side_effect.append("null")
        how_use.append("null")
        drugs.append('null')

https://www.drugs.com/abilify.html
0
12
6
Atypical antipsychotics
https://www.drugs.com/acetaminophen.html
1
12
5
Miscellaneous analgesics
https://www.drugs.com/aciphex.html
2
13
4
Proton pump inhibitors
https://www.drugs.com/actos.html
3
13
4
Thiazolidinediones
https://www.drugs.com/acyclovir.html
4
12
5
Purine nucleosides
https://www.drugs.com/adderall.html
5
13
6
CNS stimulants
https://www.drugs.com/mtm/advair-diskus.html
FROM Professional section to Overview section 
https://www.drugs.com/advair.html
6
12
5
Bronchodilator combinations
https://www.drugs.com/advil.html
7
12
6
Nonsteroidal anti-inflammatory drugs
https://www.drugs.com/albuterol.html
8
12
5
Adrenergic bronchodilators
https://www.drugs.com/monograph/albuterol-sulfate.html
FROM Professional section to Overview section 
https://www.drugs.com/albuterol.html
9
12
5
Adrenergic bronchodilators
https://www.drugs.com/aldactone.html
10
13
5
Aldosterone receptor antagonists
https://www.drugs.com/mtm/alendronate.html
11
12
4
Bisph

95
13
4
Thiazide diuretics
https://www.drugs.com/mtm/cholecalciferol.html
FROM Professional section to Overview section 
https://www.drugs.com/vitamin-d.html
96
23
6
https://www.drugs.com/cialis.html
97
12
5
Impotence agents
https://www.drugs.com/cipro.html
98
12
6
Quinolones
https://www.drugs.com/ciprofloxacin.html
99
12
5
Quinolones
https://www.drugs.com/citalopram.html
100
12
5
Selective serotonin reuptake inhibitors
https://www.drugs.com/clarithromycin.html
101
12
5
Macrolides
https://www.drugs.com/claritin.html
102
13
6
Antihistamines
https://www.drugs.com/clindamycin.html
103
12
5
Lincomycin derivatives
https://www.drugs.com/clonazepam.html
104
12
5
Benzodiazepine anticonvulsants
https://www.drugs.com/clonidine.html
105
12
4
Antiadrenergic agents, centrally acting
https://www.drugs.com/mtm/clopidogrel.html
106
13
5
Platelet aggregation inhibitors
https://www.drugs.com/mtm/clotrimazole.html
107
13
4
Azole antifungals
https://www.drugs.com/clozapine.html
108
13
4
Atypical antipsych

198
13
5
Aromatase inhibitors
https://www.drugs.com/ezetimibe.html
199
13
4
Cholesterol absorption inhibitors
https://www.drugs.com/famotidine.html
200
12
4
H2 antagonists
https://www.drugs.com/farxiga.html
201
13
5
SGLT-2 inhibitors
https://www.drugs.com/mtm/febuxostat.html
202
13
4
Antihyperuricemic agents
https://www.drugs.com/mtm/felodipine.html
203
13
4
Calcium channel blocking agents
https://www.drugs.com/femara.html
204
13
5
Aromatase inhibitors
https://www.drugs.com/fenofibrate.html
205
13
4
Fibric acid derivatives
https://www.drugs.com/fentanyl.html
206
13
4
Narcotic analgesics
https://www.drugs.com/mtm/ferrous-fumarate.html
207
13
4
Iron products
https://www.drugs.com/ferrous_sulfate.html
208
12
5
Iron products
https://www.drugs.com/mtm/fexofenadine.html
209
12
4
Antihistamines
https://www.drugs.com/mtm/finasteride.html
210
13
4
5-alpha-reductase inhibitors
https://www.drugs.com/fioricet.html
211
12
5
Analgesic combinations
https://www.drugs.com/fish_oil.html
212
12
5
Nutrace

301
13
4
Angiotensin II inhibitors with thiazides
https://www.drugs.com/mtm/hydrochlorothiazide-and-triamterene.html
302
13
4
Potassium sparing diuretics with thiazides
https://www.drugs.com/mtm/hydrochlorothiazide-and-valsartan.html
303
13
4
Angiotensin II inhibitors with thiazides
https://www.drugs.com/hydrocodone.html
304
13
5
Antitussives
https://www.drugs.com/cons/hydrocodone-and-acetaminophen.html
FROM Professional section to Overview section 
https://www.drugs.com/acetaminophen_hydrocodone.html
305
12
4
Narcotic analgesic combinations
https://www.drugs.com/monograph/hydrocodone-bitartrate.html
FROM Professional section to Overview section 
https://www.drugs.com/hydrocodone.html
306
13
5
Antitussives
https://www.drugs.com/mtm/hydrocortisone.html
307
12
5
Glucocorticoids
https://www.drugs.com/pro/hydrocortisone-cream.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/hydrocortisone-rectal-cream-ointment-suppository.html
308
12
4
Topical steroids
https://

395
13
4
Anticoagulant reversal agents
https://www.drugs.com/k-dur.html
396
13
4
Minerals and electrolytes
https://www.drugs.com/keflex.html
397
12
5
First generation cephalosporins
https://www.drugs.com/kenalog.html
398
12
5
Topical steroids
https://www.drugs.com/cons/kenalog-40.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/kenalog-40-injection.html
399
12
6
Glucocorticoids
https://www.drugs.com/keppra.html
400
12
6
Pyrrolidine anticonvulsants
https://www.drugs.com/mtm/ketalar.html
401
12
4
General anesthetics
https://www.drugs.com/mtm/ketamine.html
402
13
4
General anesthetics
https://www.drugs.com/mtm/ketoconazole.html
403
12
5
Azole antifungals
https://www.drugs.com/cdi/ketoconazole-cream.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/ketoconazole-topical.html
404
12
4
Topical antifungals
https://www.drugs.com/pro/ketoconazole-shampoo.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/ketocona

496
12
4
Miscellaneous GI agents
https://www.drugs.com/mobic.html
497
12
5
Nonsteroidal anti-inflammatory drugs
https://www.drugs.com/mtm/modafinil.html
498
13
5
CNS stimulants
https://www.drugs.com/mtm/montelukast.html
499
12
4
Leukotriene modifiers
https://www.drugs.com/morphine.html
500
12
4
Narcotic analgesics
https://www.drugs.com/motrin.html
501
12
6
Nonsteroidal anti-inflammatory drugs
https://www.drugs.com/mucinex.html
502
13
6
Expectorants
https://www.drugs.com/pro/mupirocin.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/mupirocin-topical.html
503
13
5
Topical antibiotics
https://www.drugs.com/mtm/mupirocin-topical.html
504
13
5
Topical antibiotics
https://www.drugs.com/cdi/mylanta.html
505
12
4
Antacids
https://www.drugs.com/myrbetriq.html
506
13
4
Urinary antispasmodics
https://www.drugs.com/mtm/nabumetone.html
507
13
4
Nonsteroidal anti-inflammatory drugs
https://www.drugs.com/mtm/nac.html
508
12
4
NAC
https://www.drugs.com/mtm/nadolol.html
50

597
13
5
Anorexiants
https://www.drugs.com/phenytoin.html
598
13
4
Group I antiarrhythmics
https://www.drugs.com/mtm/pioglitazone.html
599
13
4
Thiazolidinediones
https://www.drugs.com/plaquenil.html
600
13
5
Antimalarial quinolines
https://www.drugs.com/plavix.html
601
13
5
Platelet aggregation inhibitors
https://www.drugs.com/potassium_chloride.html
602
12
4
Minerals and electrolytes
https://www.drugs.com/pradaxa.html
603
12
4
Thrombin inhibitors
https://www.drugs.com/mtm/pramipexole.html
604
13
4
Dopaminergic antiparkinsonism agents
https://www.drugs.com/pravastatin.html
605
13
5
Statins
https://www.drugs.com/mtm/prazosin.html
606
13
4
Antiadrenergic agents, peripherally acting
https://www.drugs.com/mtm/prednisolone.html
607
12
5
Glucocorticoids
https://www.drugs.com/prednisone.html
608
12
5
Glucocorticoids
https://www.drugs.com/mtm/pregabalin.html
609
12
5
Gamma-aminobutyric acid analogs
https://www.drugs.com/premarin.html
610
11
5
https://www.drugs.com/prevacid.html
611
12
5
Proto

705
12
4
Miscellaneous GI agents
https://www.drugs.com/simvastatin.html
706
12
5
Statins
https://www.drugs.com/mtm/sinemet.html
707
12
5
Dopaminergic antiparkinsonism agents
https://www.drugs.com/singulair.html
708
13
5
Leukotriene modifiers
https://www.drugs.com/mtm/sitagliptin.html
709
13
5
Dipeptidyl peptidase 4 inhibitors
https://www.drugs.com/mtm/sodium-bicarbonate.html
710
12
4
Antacids
https://www.drugs.com/cons/sodium-chloride.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/sodium-chloride-flush.html
711
12
4
Minerals and electrolytes
https://www.drugs.com/mtm/solifenacin.html
712
13
4
Urinary antispasmodics
https://www.drugs.com/soma.html
713
13
4
Skeletal muscle relaxants
https://www.drugs.com/mtm/sotalol.html
714
12
5
Group III antiarrhythmics
https://www.drugs.com/spiriva.html
715
13
4
Anticholinergic bronchodilators
https://www.drugs.com/spironolactone.html
716
13
5
Aldosterone receptor antagonists
https://www.drugs.com/mtm/sprintec.html
717
1

FROM Professional section to Overview section 
https://www.drugs.com/npp/uva-ursi.html
802
17
15
https://www.drugs.com/valacyclovir.html
803
12
5
Purine nucleosides
https://www.drugs.com/mtm/valerian-root.html
804
12
2
Herbal products
https://www.drugs.com/valium.html
805
12
5
Benzodiazepine anticonvulsants
https://www.drugs.com/monograph/valproate-sodium.html
806
12
24
Valproate Sodium
https://www.drugs.com/mtm/valproic-acid.html
807
12
4
Fatty acid derivative anticonvulsants
https://www.drugs.com/mtm/valsartan.html
808
13
5
Angiotensin receptor blockers
https://www.drugs.com/valtrex.html
809
12
5
Purine nucleosides
https://www.drugs.com/vancomycin.html
810
12
5
Glycopeptide antibiotics
https://www.drugs.com/mtm/varenicline.html
811
13
4
Smoking cessation agents
https://www.drugs.com/vascepa.html
812
13
4
Miscellaneous antihyperlipidemic agents
https://www.drugs.com/vasotec.html
813
13
5
Angiotensin Converting Enzyme Inhibitors
https://www.drugs.com/veltassa.html
814
13
4
Cation excha

905
12
4
Ophthalmic antihistamines and decongestants
https://www.drugs.com/mtm/zafirlukast.html
906
13
4
Leukotriene modifiers
https://www.drugs.com/mtm/zaleplon.html
907
13
5
Miscellaneous anxiolytics, sedatives and hypnotics
https://www.drugs.com/zanaflex.html
908
13
4
Skeletal muscle relaxants
https://www.drugs.com/zantac.html
909
12
6
H2 antagonists
https://www.drugs.com/zarxio.html
910
11
5
https://www.drugs.com/zenpep.html
911
13
4
Digestive enzymes
https://www.drugs.com/zestoretic.html
912
13
4
ACE inhibitors with thiazides
https://www.drugs.com/zestril.html
913
13
5
Angiotensin Converting Enzyme Inhibitors
https://www.drugs.com/zetia.html
914
13
4
Cholesterol absorption inhibitors
https://www.drugs.com/ziac.html
915
13
4
Beta blockers with thiazides
https://www.drugs.com/cons/zidovudine.html
FROM Professional section to Overview section 
https://www.drugs.com/mtm/zidovudine-oral-injection.html
916
12
4
Nucleoside reverse transcriptase inhibitors (NRTIs)
https://www.drugs.com/mt

In [10]:
how_use[100]

['Take citalopram exactly as prescribed by your doctor. Follow all directions on your prescription label. Your doctor may occasionally change your dose. Do not use this medicine in larger or smaller amounts or for longer than recommended.Measure liquid medicine with the dosing syringe provided, or with a special dose-measuring spoon or medicine cup. If you do not have a dose-measuring device, ask your pharmacist for one.It may take up to 4 weeks before your symptoms improve. Keep using the medication as directed and tell your doctor if your symptoms do not improve.Do not stop using citalopram suddenly, or you could have unpleasant withdrawal symptoms. Ask your doctor how to safely stop using this medicine.Store at room temperature away from moisture and heat.See also: Citalopram  dosage information (in more detail)']

In [11]:
interaction[941]

['Ask a doctor or pharmacist before using  Zyrtec-D if you are also using any other drugs, including prescription and over-the-counter medicines, vitamins, and herbal products. Some medicines can cause unwanted or dangerous effects when used together. Not all possible interactions are listed in this medication guide.Taking this medicine with other drugs that make you sleepy or slow your breathing can worsen these effects. Ask your doctor before taking Zyrtec-D with a sleeping pill, narcotic pain medicine, muscle relaxer, or medicine for anxiety, depression, or seizures.See also: Zyrtec-D drug interactions (in more detail)']

In [12]:
discription[100]

['Citalopram is an antidepressant belonging to a group of drugs called selective serotonin reuptake inhibitors (SSRIs).Citalopram is used to treat depression.Citalopram may also be used for purposes not listed in this medication guide.']

In [13]:
side_effect[100]

['Get emergency medical help if you have  signs of an allergic reaction to citalopram:  skin rash or hives; difficulty breathing; swelling of your face, lips, tongue, or throat. Report any new or worsening symptoms to your doctor, such as: mood or behavior changes, anxiety, panic attacks, trouble sleeping, or if you feel impulsive, irritable, agitated, hostile, aggressive, restless, hyperactive (mentally or physically), more depressed, or have thoughts about suicide or hurting yourself. Call your doctor at once if you have:   a light-headed feeling, like you might pass out;    blurred vision, tunnel vision, eye pain or swelling, or seeing halos around lights;    headache with chest pain and severe dizziness, fainting, fast or pounding heartbeats;    severe nervous system reaction--very stiff (rigid) muscles, high fever, sweating, confusion, fast or uneven heartbeats, tremors, feeling like you might pass out;    high levels of serotonin in the body--agitation, hallucinations, fever, fas

In [14]:
drugs[100]

'Citalopram'

In [15]:
d = {'Drug_Name':drugs,'discription':discription,'drug_class':drug_class,'interaction':interaction,'side_effect':side_effect,'how_use':how_use}
df = pd.DataFrame(d)

In [154]:
df

,Drug_Name,discription,drug_class,interaction,side_effect,how_use
0,Abilify,[Abilify (aripiprazole is an antipsychotic med...,Atypical antipsychotics,[Taking Abilify with other drugs that make you...,[Get emergency medical help if you have signs...,[Take Abilify exactly as prescribed by your do...
1,Acetaminophen,[Acetaminophen is a pain reliever and a fever ...,Miscellaneous analgesics,"[Other drugs may interact with acetaminophen, ...",[Get emergency medical help if you have signs...,[Use acetaminophen exactly as directed on the ...
2,Actos,[Actos (pioglitazone) is an oral diabetes medi...,Thiazolidinediones,[Get emergency medical help if you have signs ...,[Avoid drinking alcohol. It lowers blood sugar...,[Take Actos exactly as prescribed by your doct...
3,Acyclovir,[Acyclovir is an antiviral drug. It slows the ...,Purine nucleosides,"[Acyclovir can harm your kidneys, especially i...",[Get emergency medical help if you have signs...,[Take acyclovir exactly as prescribed by your ...
4,Adderall,[Adderall contains a combination of amphetamin...,CNS stimulants,[Get emergency medical help if you have signs ...,[This medication may impair your thinking or r...,[Take Adderall exactly as prescribed by your d...
...,...,...,...,...,...,...
939,Zyprexa,[Zyprexa (olanzapine) is an antipsychotic medi...,Atypical antipsychotics,[Taking Zyprexa with other drugs that make you...,[Get emergency medical help if you have signs...,[Take Zyprexa exactly as prescribed by your do...
940,Zyrtec,[Zyrtec (cetirizine) is an antihistamine that ...,Antihistamines,[Get emergency medical help if you have signs ...,[Avoid driving or hazardous activity until you...,[Zyrtec is taken by mouth.Use Zyrtec exactly a...
941,Zyrtec-D,[Cetirizine is an antihistamine that reduces t...,Upper respiratory combinations,[Ask a doctor or pharmacist before using Zyrt...,[Get emergency medical help if you have any of...,"[Use exactly as directed on the label, or as p..."
942,Zytiga,[Zytiga (abiraterone) works by reducing androg...,Miscellaneous antineoplastics,[Get emergency medical help if you have signs ...,[Avoid eating for at least 2 hours before you ...,[Zytiga is usually taken once per day while al...


0      False
1      False
2      False
3      False
4      False
       ...  
939    False
940    False
941    False
942    False
943    False
Name: discription, Length: 944, dtype: bool
